In [ ]:
# First, import the necesary libraries and run the magic function to draw charts inline

import pandas as pd
%matplotlib inline

In [ ]:
# Load the CSV as a pandas data frame and inspect the first 5 rows

df = pd.read_csv('2015-combined-salary-seconded-utf8-en_3.csv')
df.head()

In [ ]:
# How big is this dataframe (rows, columns)
df.shape

In [ ]:
# Let's give it simple column labels for easier typing
col_names = ['sector', 'lastname', 'firstname', 'salary', 'benefits', 'employer', 'title', 'year']
df.columns = col_names
df.head()

A data frame is a two-dimensional tabular data format, much like Excel sheets, SQL tables, or R data frames. Notice the first column, with bold number, is an auto-generated index, which is quite useful. You can also assign any column as the index. More on that later.

In [ ]:
# There are two ways of accessing individual columns. As an attribute:

df.salary.head()

In [ ]:
# Or as a dict key:

df['sector'].head()

Individual columns, on the other hand, are pandas Series. These are one-dimensional data structures but that also have index labels. Data frames and Series have their own sets of methods.

In [ ]:
# To access multiple columns, use dict notation with a list of column names:
# You can also specify how many rows to show in head()
# Notice that when we have more then one column, the data structure is once again a data frame

df[['salary', 'title']].head(10)

In [ ]:
# Get some basic stats on the salary column

df.salary.describe()

In [ ]:
# Oops, pandas parsed the column as text because of the $ sign and comma
# Let's clean it up with a regex replace and turn it into a float
# Notice how pandas allows method chaining

df[['salary', 'benefits']] = df[['salary', 'benefits']].replace('[\$,]', '', regex=True).astype(float)
df.dtypes

Note about that we had to reassign the column values. If we had simply run this code:

`df[['salary', 'benefits']].replace('[\$,]', '', regex=True).astype(float)`

It would have crearted a modified copy of the columns for display. Much like a SELECT statement in SQL, it ouldn't have changed the contents of the data.

In [ ]:
# OK, now it's a number columns with real numerical stats

df.salary.describe()

In [ ]:
# That scientific notation is annoying. Let's get rid of it
# by setting float display options

pd.set_option('display.float_format', lambda x: '%.2f' % x)
df.salary.describe()

In [ ]:
# You can get the frequency of values for string columns easily

df.sector.value_counts()

In [ ]:
# How many unique employers in the dataset?

len(df.employer.unique())

In [ ]:
# We can quickly visualize the range of numerical values with a histogram

df.salary.hist(bins=20)

In [ ]:
# Default chart styles are ugly. We can import one of matplotlib's pre-loaded styles

import matplotlib as plt
plt.style.available

In [ ]:
plt.style.use('ggplot')

In [ ]:
# Now try it with a boxplot

df.salary.plot(kind='box')

In [ ]:
# Who are the highest-paid employees? Let's sort.

df.sort_values('salary', ascending=False)

In [ ]:
# Lets filter salaries over $500,000

veryrich = df[df.salary >= 500000]
veryrich.salary.describe()

A note about pandas' bizarro Boolean filtering, which is one of the most confusing parts for newcomers.

Just running the query without the brackets would have returned a Series of True and False values. That's because pandas is showing whether the condition we named is True or False for each row.

In [ ]:
df.salary >= 500000

So to only see the rows where it's True, you need that bracket notation:

In [ ]:
df[df.salary >= 500000].head(10)

In [ ]:
# Filtering by multiple conditions: wrap each condition with parenthesis.
# AND is &, OR is | (pipe)

df[(df.salary >= 500000) & (df.sector == 'Universities')]

There are many convenient methods for dealing with strings, some of which are handy for filtering. To access cells at the string level, you have to add `str` to the column name. Some examples:

In [ ]:
# Return all record with "Toronto" in the employer name

df[df.employer.str.contains('Toronto')]

In [ ]:
# Return the last names that begin with "Mc"

df[df.lastname.str.startswith('Mc')]

In [ ]:
# Take the top 10 earners and plot it on a horizontal bar chart

df.sort_values('salary', ascending=False).salary.head(10).plot(kind='barh')

In [ ]:
# Useless. That's because the plot convenience method uses the data frame index
# as labels. But we didn't set the index.
# But let's first create a new column with the full name

df['fullname'] = df['firstname'] + ' ' + df['lastname']

# Now run the plotting code again, making good use of method chaining for readability

( df.set_index('fullname')
 .sort_values('salary', ascending=False).salary
 .head(10)
 .plot(kind='barh') 
)

In [ ]:
# Indexes are very useful for filtering data by a column value

bysector = df.set_index('sector')
bysector.loc['Universities']

In [ ]:
# Or for finding the sector with the highest salary with idxmax (index of max value)

bysector.salary.idxmax()

## Grouping

In [ ]:
# Now let's calculate aggregate stats for employers. 
# pandas' grouping functions are extremely powerful. 
# The function below will calculate the max and average for each employer
# across all numerical columns.

employers = df.groupby('employer').agg(['max', 'mean'])
employers[['salary', 'benefits']]

In [ ]:
# Notice how we have two levels of column labels. This is called a multi-index column
# It has its own ways of working with it.
# For example, to sort a multi-index column, specify the levels in a tuple in a list.
# It's weird, but it makes it explicit. In this case, sort by mean salary.

employers.sort_values([('salary', 'mean')], ascending=False)

In [ ]:
# Let's calculate which employers have the highest income disparities 

# First, get only the salary column and reset the index to flatten the multi-index
emp_salaries = employers['salary'].reset_index()

# Create a new column that calculates the difference between max and mean salaries
emp_salaries['diff'] = emp_salaries['max'] - emp_salaries['mean']

# Sort by the new diff column from highest to lowest
emp_salaries.sort_values('diff', ascending = False)

In [ ]:
# You can apply a custom function to a grouped data frame.
# For example, to find the percent each sector's salaries of the whole

def calc_pct(grp):
    return grp / grp.sum() * 100

df.groupby('sector').agg({'salary': 'sum'}).apply(calc_pct)